In [18]:
import pandas as pd
import numpy as np
from plots.utils import compute_user_penalty, get_team_values_df
from common.load import load_competition_data, process_team_logs

# Import common data

In [19]:
config = 'config_vbs2023.yaml'

# load competition data from dres files and auxiliary data (FPSs, sequences)
comp_data = load_competition_data(config)

# compute team logs and put them in the form of dataframes
logs = process_team_logs(config, comp_data, force=False)

teams = comp_data['config']['teams']

/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/VBS-Analysis/VBS-2022/common/tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
/disks/workspace/Workspace/V

# Analysis

## Time Recall Table

In [20]:
def compute_data_for_time_recall_table(comp_data, teams, logs, split_user=False, only_best_user=False, max_records=10000):
    dfs = []
    for team in teams:
        team_df = logs[team].get_events_dataframe().reset_index()
        team_df = get_team_values_df(comp_data, team_df, split_user if not only_best_user else True, max_records)
        dfs.append(team_df)

    total_df = pd.concat(dfs, axis=0).reset_index()

    # keep the results from the best user among the two, for each given (task, team)
    if only_best_user:
        # the best penalty contribution is given by the ranks. If they are equal, then the user that submitted earlier wins
        user_penalty = compute_user_penalty(total_df, max_records)
                    
        total_df['user_penalty'] = user_penalty
        total_df = total_df.loc[total_df.groupby(['team', 'task'])['user_penalty'].idxmin()]
        total_df = total_df.drop(['user_penalty'], axis=1)
    
    return total_df

def build_table(df, teams):
    # drop unuseful columns
    df = df.drop(['time_first_appearance', 'rank_shot_first_appearance', 'time_last_appearance', 'rank_shot_last_appearance', 'time_first_appearance_video', 'rank_video_first_appearance'], axis=1)

    #renaming task
    rename_fun = lambda x: x.replace('vbs22-kis-t', 'T_').replace('vbs22-kis-v', 'V_')
    df['task'] = df['task'].apply(rename_fun)
    tasks = df[['task','task_start']].sort_values(by=['task_start'])['task'].unique()
    textual=[t for t in tasks if t.startswith('T')] #used later to order the column
    visual=[t for t in tasks if t.startswith('V')] #used later to order the column
    df.drop(columns='task_start', inplace=True)

    df = df.fillna(-1)
    col=[c for c in df.columns.values.tolist() if c!='team' and c!='task' and c!='user']
    df[col]= df[col].astype('int32')
    df[col] = df[col].applymap(lambda x: -1 if x< 0 else x)
    df = df.astype('str')
    df.replace(['-1'], '-', inplace=True)

    #aggregate
    agg_dic={c: (lambda x: ' / '.join(x)) for c in col}
    agg_dic['time_correct_submission']="min"
    df=df.groupby(['team','task'])[col].agg(agg_dic).reset_index()
    df.replace('- / -', '-', regex=True, inplace=True)
    add_second= lambda x: x if x=='-' else x + 's'
    df['time_correct_submission']=df['time_correct_submission'].apply(add_second)
    df['time_best_shot']=df['time_best_shot'].apply(add_second)
    df['rank_shot_margin_5'] = df['rank_shot_margin_5'].apply(add_second)
    df['time_best_video'] = df['time_best_video'].apply(add_second)

    df = df.melt(var_name="metric", id_vars=["team", "task"], value_name="value")
    df['unit'] = df['metric'].apply(lambda x: 'rank' if x.startswith('rank_') else 'time')
    replace_dic = {
        'rank_shot_margin_0': 'correct frame',
        'time_best_shot': 'correct frame',
        'rank_shot_margin_5': 'frame in GT+2x5s',
        'time_best_shot_margin5': 'frame in GT+2x5s',
        'rank_video': 'correct video',
        'time_best_video': 'correct video',
        'time_correct_submission': 'correct submission'
    }
    df['metric']=df['metric'].map(replace_dic)
    df = df.pivot(index=['team', 'metric', 'unit'], columns="task", values="value")
    df=df.fillna('-')


    #sorting index desired order
    level_0=teams #order in the conf file
    level_1=['correct frame', 'frame in GT+2x5s', 'correct video','correct submission']
    level_2=['rank','time']
    df = df.reindex(pd.MultiIndex.from_product([level_0,level_1,level_2]))
    df.dropna(axis=0, inplace=True)#'correct submission'/rank shluld not be in the index
    df_margin_5 = df.copy()

    # sorting index desired order
    level_0 = teams  # order in teh conf file
    level_1 = ['correct frame', 'correct video', 'correct submission']
    level_2 = ['rank', 'time']
    df = df.reindex(pd.MultiIndex.from_product([level_0, level_1, level_2]))
    df.dropna(axis=0, inplace=True)  # 'correct submission'/rank shluld not be in the index

    #ordering columns
    df=df[textual+visual]

    return df, df_margin_5

In [21]:
df = compute_data_for_time_recall_table(comp_data, teams, logs)
table, table5 = build_table(df, teams)
table5

task                            vbs23-kis-t1 vbs23-kis-t2 vbs23-kis-t3  \
VISIONE correct frame      rank            1            1            1   
                           time          26s          65s         384s   
        frame in GT+2x5s   rank           1s           1s           1s   
                           time           26           65          384   
        correct video      rank            1            1            1   
                           time          26s          59s         384s   
        correct submission time          65s          76s            -   

task                            vbs23-kis-t4 vbs23-kis-t5 vbs23-kis-t6  \
VISIONE correct frame      rank           16            1            1   
                           time          76s          88s          23s   
        frame in GT+2x5s   rank          16s           1s           1s   
                           time           76           88           23   
        correct video      rank           16            1            1   
                           time          76s          88s          23s   
        correct submission time         111s         100s          52s   

task                            vbs23-kis-t7 vbs23-kis-v-m1 vbs23-kis-v-m2  \
VISIONE correct frame      rank            4              6             80   
                           time          64s            23s           205s   
        frame in GT+2x5s   rank           4s             4s            80s   
                           time           64             23            205   
        correct video      rank            4              4             80   
                           time          64s            23s           205s   
        correct submission time          95s            76s           254s   

task                            vbs23-kis-v-m3 vbs23-kis-v-m4 vbs23-kis-v-m5  \
VISIONE correct frame      rank             17             33             27   
                           time            12s           245s            19s   
        frame in GT+2x5s   rank            17s            33s            27s   
                           time             12            245             19   
        correct video      rank             17             33             27   
                           time            12s           245s            19s   
        correct submission time            27s           266s            51s   

task                            vbs23-kis-v-m7 vbs23-kis-v1 vbs23-kis-v2  \
VISIONE correct frame      rank              2            4         3752   
                           time            37s          16s          46s   
        frame in GT+2x5s   rank             1s           1s         106s   
                           time             12           16           87   
        correct video      rank              1            1          106   
                           time            12s          16s          87s   
        correct submission time            44s          25s         112s   

task                            vbs23-kis-v3 vbs23-kis-v4 vbs23-kis-v5  \
VISIONE correct frame      rank            2           24           19   
                           time          26s          18s          62s   
        frame in GT+2x5s   rank           2s          24s          19s   
                           time           26           18           62   
        correct video      rank            1           24            1   
                           time          26s          18s          65s   
        correct submission time          34s          25s          72s   

task                            vbs23-kis-v6  
VISIONE correct frame      rank          224  
                           time         298s  
        frame in GT+2x5s   rank         224s  
                           time          298  
        correct video      rank            2  
                           time         163s  
        correct s